# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [197]:
# Data manipulation
import cufflinks as cf
import altair as alt
from IPython import get_ipython
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
InteractiveShell.ast_node_interactivity = 'all'

ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
alt.renderers.enable('notebook')
alt.renderers.enable('html')

cf.go_offline(connected=True)
cf.set_config_file(theme='white')

RendererRegistry.enable('notebook')

RendererRegistry.enable('html')

# Data Wrangling

In [198]:
# Reading in data
demographics = pd.read_csv("dataScienceTask/T_demo.csv")
dbp = pd.read_csv("dataScienceTask/T_DBP.csv")
sbp = pd.read_csv("dataScienceTask/T_SBP.csv")
creatinine = pd.read_csv("dataScienceTask/T_creatinine.csv")
glucose = pd.read_csv("dataScienceTask/T_glucose.csv")
meds = pd.read_csv("dataScienceTask/T_meds.csv")
hgb = pd.read_csv("dataScienceTask/T_HGB.csv")
ldl = pd.read_csv("dataScienceTask/T_ldl.csv")
stage = pd.read_csv("dataScienceTask/T_stage.csv")

In [199]:
# Aggregating data into 1 dataframe
df = pd.DataFrame()

for i in demographics['id']:
    temp = dbp.loc[dbp['id'] == i][['time', 'value']].merge(sbp.loc[sbp['id'] == i][[
        'value', 'time']], left_on='time', right_on='time', suffixes=('_dbp', '_sbp'), how='outer')
    temp = temp.merge(creatinine.loc[creatinine['id'] == i][['time', 'value']], left_on='time',
                      right_on='time', how='outer').rename(columns={"value": "value_creatinine"})
    temp = temp.merge(glucose.loc[glucose['id'] == i][['time', 'value']], left_on='time',
                      right_on='time', how='outer').rename(columns={"value": "value_glucose"})
    temp = temp.merge(hgb.loc[hgb['id'] == i][['time', 'value']], left_on='time',
                      right_on='time', how='outer').rename(columns={"value": "value_hgb"})
    temp = temp.merge(ldl.loc[ldl['id'] == i][['time', 'value']], left_on='time',
                      right_on='time', how='outer').rename(columns={"value": "value_ldl"})
    temp = temp.sort_values(by=['time'])
    temp['id'] = i
    df = df.append(temp)

df = df.merge(demographics[['id', 'gender', 'race']],
              left_on='id', right_on='id')

df

,time,value_dbp,value_sbp,value_creatinine,value_glucose,value_hgb,value_ldl,id,gender,race
0,0,95.32,134.11,1.29,6.24,13.51,161.49,0,Male,Unknown
1,98,83.98,133.75,NaN,NaN,NaN,NaN,0,Male,Unknown
2,107,NaN,NaN,1.15,7.20,13.39,111.39,0,Male,Unknown
3,279,65.97,125.08,NaN,NaN,NaN,NaN,0,Male,Unknown
4,286,NaN,NaN,1.44,7.01,12.84,NaN,0,Male,Unknown
...,...,...,...,...,...,...,...,...,...,...
4756,621,72.52,113.38,NaN,NaN,NaN,NaN,299,Male,Asian
4757,652,NaN,NaN,0.82,NaN,NaN,NaN,299,Male,Asian
4758,659,NaN,NaN,NaN,6.20,14.43,79.02,299,Male,Asian
4759,1069,NaN,NaN,NaN,NaN,14.90,NaN,299,Male,Asian


In [200]:
df.describe()

,time,value_dbp,value_sbp,value_creatinine,value_glucose,value_hgb,value_ldl,id
count,4761.00000,1821.000000,1809.000000,1439.000000,1556.000000,2025.000000,1261.000000,4761.000000
mean,462.41924,79.518583,134.155368,1.328999,6.694833,13.825185,87.558795,148.060912
std,334.49964,10.178676,14.765817,0.357971,1.599058,1.644946,28.156104,88.343602
min,0.00000,44.950000,91.990000,0.240000,2.890000,8.820000,26.100000,0.000000
25%,196.00000,72.830000,124.630000,1.080000,5.620000,12.670000,67.290000,70.000000
50%,414.00000,79.050000,133.440000,1.290000,6.360000,13.920000,83.960000,147.000000
75%,640.00000,86.170000,143.160000,1.530000,7.440000,14.980000,105.140000,227.000000
max,1429.00000,112.930000,211.090000,3.020000,16.610000,19.000000,198.590000,299.000000


In [201]:
demo_stage = demographics.merge(
    stage, left_on='id', right_on='id', how='outer')

In [202]:
aggregate = pd.DataFrame()

In [203]:
for i in demographics['id']:
    dbp = df[(df['id'] == i) & (df['value_dbp'] > 85)
             ]['value_dbp'].count() / df[(df['id'] == i)]['value_dbp'].count()
    sbp = df[(df['id'] == i) & (df['value_sbp'] > 129)
             ]['value_sbp'].count() / df[(df['id'] == i)]['value_sbp'].count()
    hgb = df[(df['id'] == i) & (df['value_hgb'] < 9)
             ]['value_hgb'].count() / df[(df['id'] == i)]['value_hgb'].count()
    ldl = df[(df['id'] == i) & (df['value_ldl'] > 160)
             ]['value_ldl'].count() / df[(df['id'] == i)]['value_ldl'].count()
    glucose = df[(df['id'] == i) & (df['value_glucose'] > 6.9)
                 ]['value_glucose'].count() / df[(df['id'] == i)]['value_glucose'].count()
    if demographics[demographics['id'] == i]['gender'].item() == 'Male':
        creatinine = df[(df['id'] == i) & (df['value_creatinine'] > 1.2)]['value_creatinine'].count(
        ) / df[(df['id'] == i)]['value_creatinine'].count()
    else:
        creatinine = df[(df['id'] == i) & (df['value_creatinine'] > 1.0)]['value_creatinine'].count(
        ) / df[(df['id'] == i)]['value_creatinine'].count()

    temp = pd.DataFrame({'id': i, 'gender': demographics[demographics['id'] == i]['gender'],
                        'dbp': dbp, 'sbp': sbp, 'hgb': hgb, 'ldl': ldl, 'glucose': glucose, 'creatinine': creatinine})
    aggregate = aggregate.append(temp)

aggregate = aggregate.merge(
    demo_stage[['race', 'age', 'Stage_Progress', 'id']], left_on='id', right_on='id', how='inner')

In [204]:
pd.set_option('display.max_rows', None)
aggregate

,id,gender,dbp,sbp,hgb,ldl,glucose,creatinine,race,age,Stage_Progress
0,0,Male,0.375000,0.875000,0.000,0.333333,0.333333,0.666667,Unknown,70,True
1,1,Female,0.285714,1.000000,0.000,0.000000,1.000000,1.000000,White,71,False
2,2,Female,0.750000,1.000000,0.000,0.000000,0.666667,1.000000,White,57,True
3,3,Male,0.000000,0.714286,0.000,0.000000,0.333333,0.200000,White,74,False
4,4,Female,0.857143,0.857143,0.000,0.000000,1.000000,0.833333,White,50,True
5,5,Male,0.000000,0.250000,0.000,0.000000,0.000000,1.000000,Black,73,False
6,6,Female,0.000000,0.600000,0.000,0.000000,0.750000,1.000000,White,60,False
7,7,Female,0.666667,0.333333,0.000,0.000000,0.000000,1.000000,Asian,62,False
8,8,Female,0.666667,1.000000,0.000,0.000000,0.000000,1.000000,White,68,True
9,9,Female,0.000000,0.875000,0.000,0.000000,0.000000,1.000000,White,80,False


In [205]:
demo_stage

,id,race,gender,age,Stage_Progress
0,0,Unknown,Male,70,True
1,1,White,Female,71,False
2,2,White,Female,57,True
3,3,White,Male,74,False
4,4,White,Female,50,True
5,5,Black,Male,73,False
6,6,White,Female,60,False
7,7,Asian,Female,62,False
8,8,White,Female,68,True
9,9,White,Female,80,False


In [206]:
demo_stage

,id,race,gender,age,Stage_Progress
0,0,Unknown,Male,70,True
1,1,White,Female,71,False
2,2,White,Female,57,True
3,3,White,Male,74,False
4,4,White,Female,50,True
5,5,Black,Male,73,False
6,6,White,Female,60,False
7,7,Asian,Female,62,False
8,8,White,Female,68,True
9,9,White,Female,80,False


# Basic Data Visualization of the Demographics
Looking at basic demographics of the patients

In [207]:
alt.Chart(demo_stage).mark_bar().encode(
    alt.X("age:Q", bin=True),
    alt.Y('count()', title='Number of Patients'),
    column='race',
    color='Stage_Progress'
).properties(
    width=150,
    height=120
)

alt.Chart(...)

In [208]:
alt.Chart(demo_stage).mark_bar().encode(
    alt.X("age:Q", bin=True),
    alt.Y("count()", title='Number of Patients'),
    column='gender',
    color='Stage_Progress'
).properties(
    width=300,
    height=150
)

alt.Chart(...)

Looking at the demographics data, there are more white patients than of the other races. In terms of gender, there are more more female patients across all age bins, but the difference is not as large as compared to the race differences. 

In [209]:
alt.Chart(demo_stage, title='Percentage of CKD patients').mark_bar().encode(
    alt.X("age:Q", bin=True),
    alt.Y('count()', stack="normalize", axis=alt.Axis(
        format=".0%"), title='Precentage of Patients'),
    color='Stage_Progress'
).properties(
    width=300,
    height=150
)

alt.Chart(...)

Just looking at outcomes (Stage_Progress), there are more patients who do not have chronic kidney disease (CKD) compared to patients who have CKD. 

In [210]:
positive = alt.Chart(demo_stage.loc[demo_stage['Stage_Progress'] == True], title="CKD Positive").mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('age:Q', bin=True),
    y=alt.Y('count():Q', scale=alt.Scale(
        domain=(0, 46)), title='Number of Patients'),
    color='race:N'
)
negative = alt.Chart(demo_stage.loc[demo_stage['Stage_Progress'] == 0], title="CKD Negative").mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('age:Q', bin=True),
    y=alt.Y('count():Q', scale=alt.Scale(
        domain=(0, 46)), title='Number of Patients'),
    color='race:N'
)

positive | negative

alt.HConcatChart(...)

In [211]:
male = alt.Chart(demo_stage.loc[demo_stage['gender'] == 'Male'], title="Male").mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('age:Q', bin=True),
    y=alt.Y('count():Q', scale=alt.Scale(
        domain=(0, 46)), title='Number of Patients'),
    color='race:N'
)

female = alt.Chart(demo_stage.loc[demo_stage['gender'] == 'Female'], title="Female").mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('age:Q', bin=True),
    y=alt.Y('count():Q', scale=alt.Scale(
        domain=(0, 46)), title='Number of Patients'),
    color='race:N'
)

male | female

alt.HConcatChart(...)

# Analysis/Modeling
Do work here

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here